## 스트리밍 토큰 수 계산하기 (OpenAI, LangChain)

### 1. OpenAI

이제 OpenAI 스트리밍 모드에서도 토큰 수를 확인할 수 있습니다.

-- **2024.4 업데이트: [OpenAI Cookbook 링크](https://cookbook.openai.com/examples/how_to_stream_completions#4-how-to-get-token-usage-data-for-streamed-chat-completion-response)**

In [ ]:
!pip install openai langchain langchain_openai langchain_community --upgrade -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.7 MB/s eta 0:00:00


In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "여기에 API 키 입력"
client = openai.Client()

`stream_options={"include_usage": True}`를 추가하면 마지막 청크에 사용량 정보가 전달됩니다.

In [ ]:
topic = 'Potatoes'
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': f"""tell me a english joke about {topic},
also, explain in Korean why it is fun for english-speakers.
Include direct translation of the joke."""}
    ],
    temperature=0.2,
    stream=True, # 스트리밍 옵션 추가
    stream_options={"include_usage": True}
    # 맨 마지막 청크에 사용량 정보를 포함
)

for chunk in response:
    if chunk.choices!=[]:
        token = chunk.choices[0].delta.content
        if token: print(token,end='',flush=True)

print('\n\n')
print(chunk.usage)

Sure! Here's a classic English joke about potatoes:

**Joke:**
Why do potatoes make good detectives?
Because they keep their eyes peeled!

**Direct Translation:**
왜 감자는 좋은 탐정이 될까요?
왜냐하면 그들은 눈을 벗기고 있기 때문이에요!

**Explanation in Korean:**
이 농담은 영어 단어의 중의성을 이용한 것입니다. "Keep their eyes peeled"라는 표현은 "눈을 벗기다"라는 직역과 "눈을 크게 뜨고 주의 깊게 보다"라는 관용적 의미를 동시에 가지고 있습니다. 감자는 껍질을 벗길 때 눈(싹)이 드러나기 때문에, 이 표현이 감자와 관련된 농담으로 사용된 것입니다. 영어 사용자에게는 이중 의미가 재미있게 다가옵니다.


CompletionUsage(completion_tokens=178, prompt_tokens=39, total_tokens=217)


<br><br>
<br><br>

### 2. LangChain
랭체인도 같은 방식으로 `stream_options={"include_usage": True}` 를 통해 토큰 수를 구할 수 있습니다.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
prompt = ChatPromptTemplate.from_template("""tell me a english joke about {topic},
                also, explain in Korean why it is fun for english-speakers.
                Include direct translation of the joke.""")


In [ ]:
result = llm.stream(prompt.format(topic='walnet'), stream_options={"include_usage": True})
for chunk in result:
    print(chunk.content, end='', flush=True)

print('\n\n')
print(chunk.usage_metadata)

English joke: Why did the walnut go to the party? Because it was a little nutty!

Korean explanation: 영어권 사람들에게 재미있는 이유는 "nutty"가 "조금 미친"이라는 뜻이 있어서, walnut이 조금 미친 것처럼 파티에 가서 재미있게 보낸다는 농담입니다.

Direct translation: "호두가 왜 파티에 갔나요? 약간 미친 것 같아서요!"


{'input_tokens': 43, 'output_tokens': 146, 'total_tokens': 189}


<br><br>
만약 LCEL Chain에서 이를 활용하고 싶다면, 아래와 같이 설정해줄 수 있습니다.

In [ ]:
llm2 = ChatOpenAI(
    model="gpt-4o",
    model_kwargs={"stream_options": {"include_usage": True}},
)

chain2 = prompt | llm2

In [ ]:
result = chain2.stream({'topic':'chickens'})
for chunk in result:
    print(chunk.content, end='', flush=True)

print('\n\n')
print(chunk.usage_metadata)

**Joke:**
Why did the chicken join a band?
Because it had the drumsticks!

**Direct Translation:**
왜 닭이 밴드에 들어갔을까요?
왜냐하면 닭다리가 있었기 때문이에요!

**Explanation in Korean:**
이 농담은 영어권 사람들에게 재미있는 이유는 'drumsticks'라는 단어에 있습니다. 'Drumsticks'는 두 가지 의미를 가지고 있어요. 첫째, 드럼을 칠 때 사용하는 드럼 스틱을 의미하고, 둘째, 닭다리를 의미합니다. 그래서 이 농담은 닭이 '드럼 스틱'을 가지고 있기 때문에 밴드에 들어갔다는 두 가지 의미를 가진 말장난입니다. 이러한 언어 유희가 영어권 사람들에게 웃음을 줍니다.


{'input_tokens': 40, 'output_tokens': 175, 'total_tokens': 215}


<br><br>
<br><br>

### 3. LangChain Callback을 이용한 방법   
기존의 `StdOutCallbackHandler`는 스트리밍 환경에서 동작하지 않기 때문에, 비슷한 콜백을 만들어서 활용할 수도 있습니다.


In [ ]:
from langchain_core.callbacks import BaseCallbackHandler
from typing import Dict, Any, List
import tiktoken

from langchain_community.callbacks.openai_info import get_openai_token_cost_for_model
# 모델별 Cost를 계산하는 함수

model_name = 'gpt-4o'


enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

class TokenMetricsCallbackHandler(BaseCallbackHandler):

    prompt_tokens = 0
    completion_tokens = 0
    completion_cost = 0
    prompt_cost = 0
    total_cost = 0.0

    def __repr__(self) -> str:
        return (
            f"Tokens Used: {self.prompt_tokens + self.completion_tokens}\n"
            f"\tPrompt Tokens: {self.prompt_tokens}\n"
            f"\tCompletion Tokens: {self.completion_tokens}\n"
            f"Total Cost (USD): ${self.total_cost:f}\n"
            f"\tPrompt Cost (USD): ${self.prompt_cost:f}\n"
            f"\tCompletion Cost (USD): ${self.completion_cost:f}"

        )

    async def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any) -> None:
        """프롬프트 토큰 개수 계산"""
        self.prompt_tokens += len(enc.encode(prompts[0]))

    async def on_llm_new_token(self, token: str, **kwargs):
        """출력 토큰 개수 1씩 증가"""
        self.completion_tokens += 1

    async def on_llm_end(self, response, **kwargs):
        """OpenAI 모델 이름으로부터 토큰 비용 계산"""
        self.prompt_cost += get_openai_token_cost_for_model(model_name, self.prompt_tokens)
        self.completion_cost += get_openai_token_cost_for_model(model_name, self.completion_tokens, is_completion=True)
        self.total_cost = self.prompt_cost + self.completion_cost


callback_handler = TokenMetricsCallbackHandler()

config = {
    'callbacks':[callback_handler]
}



prompt = ChatPromptTemplate.from_template("""tell me a english joke about {topic},
                also, explain in Korean why it is fun for english-speakers.
                Include translation of the joke.""")

chain = prompt | llm
result = chain.stream({'topic':'Ice Cream'}, config=config)
for chunk in result:
    print(chunk.content,end='',flush=True)

print('\n\n')
print(callback_handler)

Joke: Why did the ice cream break up with the sundae? Because it couldn't find a cone-nection!

Explanation in Korean: 영어권 사람들에게 이 농담이 재미있는 이유는 "cone-nection"이라는 말장난 때문입니다. "Cone-nection"은 "connection"(연결)과 "cone"(콘)을 합쳐 만든 신조어로, 아이스크림이 선데이와 헤어진 이유를 연결고리(연결)를 찾지 못해서라는 농담입니다.

Translation: 아이스크림이 선데이와 헤어진 이유는 뭘까요? 콘과 연결고리를 찾지 못해서였어요!


Tokens Used: 199
	Prompt Tokens: 35
	Completion Tokens: 164
Total Cost (USD): $0.002635
	Prompt Cost (USD): $0.000175
	Completion Cost (USD): $0.002460
